[Collab Notebook Link](https://colab.research.google.com/drive/1kntdWPZ0uxYSIqCV4yCB-Wb_Qpyy7rYi?usp=sharing)

##"Classical" Tabular Q-Learning Implementation of Tic Tac Toe

**Introduction**

In this homework, I implemeented a table based Q-learning Agent that would learn to play tic tac toe. Given a table with all possible game states and actions that could be peformed in a game, the algorithm plays games and attempts to learn an optimal strategy based on ε-greedy action selection which starts if off by selecting random actions but gradually causes the algorithm to focus on a fixed strategy.

While the tic tac toe board is typically thought of as a 3x3 board, it can be flattened to a 1 dimensional array of length 9 and still be "played" properly. I take advantage of this to work primarily with 1D arrays for the purposes of this assignment as it simplifies tasks like making a straightfoward lookup table for game states and the indexes they correspond to on a 2d Qtable represented by a 2d numpy array.

To evaluate the success of our Q Learning Agent, I tested it in the following scenarios - playing games vs a random agent (both in the player 1 and player 2 seats), and having 2 Q Agents play versus each other. The agents are evaluated based on their performance as Player 1, Player 2, and the number of draws they incurred in a match, as well as a more qualitative metric of whether they perform more "human" tic tac toe strategies. Some of the games from each episodes are presented as part of their observations, and can be found under each training situation's output.

Player 1 is represented by X (1), and Player 2 is represented by O (2). Empty spots are marked by E (zero). 
 
**Resources**

**Base Q Learning Resources**

https://towardsdatascience.com/practical-reinforcement-learning-02-getting-started-with-q-learning-582f63e4acd9

https://github.com/Uzmamushtaque/CSCI4962-Projects-ML-AI/blob/main/Lecture_18.ipynb

These were the base Q Learning resources provided at the start of the assignemnt. These were used to gain a basic understanding of how to implement Q-learning for a problem, specifically using the provided article's pseudo-code as a starting point.

**Studying Q Learning Via the Frozen Lake Problem**

https://www.freecodecamp.org/news/diving-deeper-into-reinforcement-learning-with-q-learning-c18d0db58efe/

https://github.com/simoninithomas/Deep_reinforcement_learning_Course/blob/master/Q%20learning/FrozenLake/Q%20Learning%20with%20FrozenLake.ipynb

I then tried a basic implementation using the Frozen Lake Problem availalbe in openAI gym to get a better understanding of what a Q Learning Agent would look like when learning to play a game.

**Understanding the Specifics of the Tic Tac Toe Problem**

https://medium.com/@carsten.friedrich/part-1-computer-tic-tac-toe-basics-35964f92fa03

https://medium.com/@carsten.friedrich/part-3-tabular-q-learning-a-tic-tac-toe-player-that-gets-better-and-better-fa4da4b0892a

https://nestedsoftware.com/2019/07/25/tic-tac-toe-with-tabular-q-learning-1kdn.139811.html

After initial research and experimenting with openAI gym, I implemented the Tic Tac Toe game with the interest of making it possible to create multiple different agents that could play the game against each other as part of the training process. This ended up being a slightly ambitious demand for the time available, so I scaled the problem down to implementing Tic Tac Toe with a Random Agent first to ensure I was properly setting up a simulated game that could support modular agents  prior to implementing a Q Agent.

Additional attribution is provided where appropriate.

#Base Game
Here we develop the base tictactoe game and an agent playing random moves to accompany it.

In [61]:
import numpy as np
from enum import Enum
from abc import ABC, abstractmethod
import random

In [62]:
class MARKERS(Enum):
    E = 0 
    X = 1
    O = 2 

class GAMESTATE(Enum):
    ONGOING = 0 
    PLAYER_1 = 1
    PLAYER_2 = 2
    DRAW = 3 
    ERROR = 4 

In [63]:
print(MARKERS.E.value)

0


In [64]:
"""
A tic tac toe board looks like a 3x3 grid:
0 1 2 
3 4 5
6 7 8

But we can reinterpret it as a 1 dimensional array of size 9.
0 1 2 3 4 5 6 7 8

If we use 1 for X and 2 for O, we can reinterpret winning positions accordingly:
column: That column, offset by row#*3
altenratively, 0=3=6 OR 1=4=7 OR 2=5=8
1 0 0 
1 0 0
1 0 0

1 0 0 1 0 0 1 0 0

row: Indexes 0=1=2 OR 3=4=5 OR 6=7=8
1 1 1
0 0 0
0 0 0

1 1 1 0 0 0 0 0 0

diagonal: 0=4=8 or 2=4=6
1 0 0
0 1 0
0 0 1
1 0 0 0 1 0 0 0 1
0 0 1 0 1 0 1 0 0

We could do more graceful calculations for these
but for simplicity's  sake we will do simple index checks
"""

class TicTacToe:
    def __init__(self, base_state=np.zeros(shape=(9), dtype=int)):
            self.state = base_state.copy()

    def reset_board(self):
      self.state = np.zeros(shape=(9), dtype=int)
    def printstate(self):
      print(self.state[0:3]) #[start index:end index+1]
      print(self.state[3:6])
      print(self.state[6:9])

    #print as xs and os instead
    def printstate_xo(self):
      for i in self.state[0:3]:
        print(MARKERS(i).name, end=' ')
      print(' ')
      for i in self.state[3:6]:
        print(MARKERS(i).name, end=' ')
      print(' ')
      for i in self.state[6:9]:
        print(MARKERS(i).name, end=' ')
      print(' ')

    def getval(self, index):
      return self.state[index]

    #checks for a winner. If there is a winner,
    #return the winner's number.
    #otherwise, return 0 to indicate no winner found.
    def check_row(self):
      if(self.state[0] == self.state[1] == self.state[2]):
        return self.state[0]
      elif(self.state[3] == self.state[4] == self.state[5]):
        return self.state[3]
      elif(self.state[6] == self.state[7] == self.state[8]):
        return self.state[6]
      return 0

    def check_col(self):
      if(self.state[0] == self.state[3] == self.state[6]):
        return self.state[0]
      elif(self.state[1] == self.state[4] == self.state[7]):
        return self.state[1]
      elif(self.state[2] == self.state[5] == self.state[8]):
        return self.state[2]
      return 0

    def check_diag(self):
      if(self.state[0] == self.state[4] == self.state[8]):
        return self.state[0]
      elif(self.state[2] == self.state[4] == self.state[6]):
        return self.state[2]
      return 0

    #the game should end if we pass a nonzero result.
    def victory_check(self):
      row=self.check_row()
      col=self.check_col()
      diag=self.check_diag()
      if row !=0:
        return row
      elif col !=0:
        return col
      elif diag !=0:
        return diag
      elif(np.all(self.state)): #np.all will return false if a 0 is found in the array
        return 3 #this is a special condition to indicate a draw
      return 0 #0 indicates the game is still ongoing
      
    #converts (column, row) coordinates passed as a tuple to a 1D coord
    #assumes 0,0 is the top left corner
    #formula is row*3 + col
    #e.g. 
    #0,0 = 0
    #1,2 = 7
    def twod_to_oned(self, coord):
      return (coord[1] * 3) + coord[0]

    #return 1 if placement was successful 
    #else return -1 if already occupied slot
    def place1d(self, value, index):
      if(self.state[index]==0):
        self.state[index] = value
        return 1
      return -1
    def place2d(self, value, coord):
      return self.place1d(value, self.twod_to_oned(coord))

    #get all empty indices
    def getempty(self):
      temp_empty = []
      for i in range(0,9):
        if self.state[i] == 0:
          temp_empty.append(i)
      return temp_empty



    

In [65]:
a = TicTacToe()
#print(a.state.size)
a.printstate()

[0 0 0]
[0 0 0]
[0 0 0]


In [66]:
a.place1d(1,4)
a.place2d(2,(1,2))
a.place1d(42,4) #should fail
a.printstate()
a.printstate_xo()
print("reset test")
a.reset_board()
a.printstate()

[0 0 0]
[0 1 0]
[0 2 0]
E E E  
E X E  
E O E  
reset test
[0 0 0]
[0 0 0]
[0 0 0]


In [67]:
#test victory
a.place2d(2,(0,0))
a.place2d(2,(1,1))
a.place2d(2,(2,2))
a.printstate()
print(a.victory_check())
a.reset_board()

[2 0 0]
[0 2 0]
[0 0 2]
2


#Random Moves Agent
As a preparatory step to making a Q-Learning Agent we first make an agent
that only knows how to pick random moves.

In [68]:
#abstract class that will define all future agents
class Agent(ABC):
    @abstractmethod
    def move(self):
      pass

class RandomAgent(Agent):
  def __init__(self, side):
    self.side = side
  def move(self, tictactoe):
    empty = tictactoe.getempty()
    tictactoe.place1d(self.side, random.choice(empty))

In [69]:
randage1 = RandomAgent(1)
randage2 = RandomAgent(2)

In [70]:
a.reset_board()
randage1.move(a)
randage2.move(a)
randage1.move(a)
randage2.move(a)
a.printstate()

[2 1 2]
[0 0 1]
[0 0 0]


In [71]:
#let's try and make a sample game
gameend = False
result = 4 
a.reset_board()
while (gameend == False):
  randage1.move(a)
  vict = a.victory_check() 
  if vict !=0:
    result = vict
    gameend = True
    break
  randage2.move(a)
  vict = a.victory_check() 
  if vict !=0:
    result = vict
    gameend = True
    break

a.printstate()
a.printstate_xo()
print("Game is over. Resulting state: ", GAMESTATE(result).name)

[1 0 2]
[2 1 2]
[1 0 1]
X E O  
O X O  
X E X  
Game is over. Resulting state:  PLAYER_1


#Random Agent Games

In this context, we have 2 random agents play vs each other.

In [148]:
#let's try 200 games 
#player1wins,player2wins,drawwins, errors
results_array = [0,0,0,0]

for i in range (0,201):

  gameend = False
  result = 4 
  a.reset_board()
  if(i%100 == 0):
    print("Game of Episode ", i)
  while (gameend == False):
    randage1.move(a)
    vict = a.victory_check() 
    if(i%100 == 0):
      print("P1")
      a.printstate_xo()
    if vict !=0:
      result = vict
      gameend = True
      break
    randage2.move(a)
    if(i%100 == 0):
      print("P2")
      a.printstate_xo()
    vict = a.victory_check() 
    if vict !=0:
      result = vict
      gameend = True
      break
  results_array[result-1]+=1

print("Game Statistics: P1 Win, P2 Win, Draw, Error")  
print(results_array)


Game of Episode  0
P1
X E E  
E E E  
E E E  
P2
X E E  
E E E  
E E O  
P1
X E E  
X E E  
E E O  
P2
X E E  
X E O  
E E O  
P1
X E X  
X E O  
E E O  
P2
X O X  
X E O  
E E O  
P1
X O X  
X E O  
E X O  
P2
X O X  
X E O  
O X O  
P1
X O X  
X X O  
O X O  
Game of Episode  100
P1
X E E  
E E E  
E E E  
P2
X E E  
E E E  
E O E  
P1
X E E  
E E E  
X O E  
P2
X E E  
E E O  
X O E  
P1
X E E  
E X O  
X O E  
P2
X E E  
E X O  
X O O  
P1
X E E  
X X O  
X O O  
Game of Episode  200
P1
E E X  
E E E  
E E E  
P2
O E X  
E E E  
E E E  
P1
O X X  
E E E  
E E E  
P2
O X X  
E E E  
O E E  
P1
O X X  
E X E  
O E E  
P2
O X X  
E X O  
O E E  
P1
O X X  
E X O  
O X E  
Game Statistics: P1 Win, P2 Win, Draw, Error
[125, 50, 26, 0]


#Q Learning Agent

We will now build a Q Learning agent whose goal is to construct a Q table that it can use to play the game. 


https://stackoverflow.com/questions/7466429/generate-a-list-of-all-unique-tic-tac-toe-boards

It is worth noting that there are 19683 possible states in a tic tac toe game.

In [ ]:
"""
https://towardsdatascience.com/practical-reinforcement-learning-02-getting-started-with-q-learning-582f63e4acd9

PseudoCode of Q Learning Algorithm

Define Gamma, Epilson, Alpha
Initiate q-table with zeroes
observe initial state s
repeat:
  select action a
    with probability epsilon select random action a
    else selection action a = argmax(Q(s,a'))
  carry out action a
  observe next state s' and reward r
  calculate Q_target = r+gamma*max[Q(s',A)]
  calculate Q_delta = Q_target - Q(s,a)
  add alpha*Q_delta to Q(s,a)
  s=s'
"""

In [74]:
#Game state generator
#Attribution: https://stackoverflow.com/questions/7466429/generate-a-list-of-all-unique-tic-tac-toe-boards
#Quick generation of all possible tic tac toe gamestates 
#(regardless of legality)
#for the purposes of having the "state" indexes 
#of the q table correspond to these game states.
def generateStates():
  StatesMatrix = np.zeros((3**9,9))
  for i in range(3**9):
      c = i
      for j in range(9):
        StatesMatrix[i][j] = c % 3
        c //= 3
  return StatesMatrix

In [167]:
def searchStates(statesmat, target):
  for i in range(0, statesmat.shape[0]):
    if np.array_equal(statesmat[i], target):
      return i

In [166]:
statesm = generateStates()
print(statesm)
print(statesm.shape)
print(statesm.shape[0])

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 ...
 [0. 2. 2. ... 2. 2. 2.]
 [1. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]]
(19683, 9)
19683


In [76]:
print(searchStates(statesm,np.array([0., 0., 0., 0., 0., 0., 0., 0., 0.])))
print(searchStates(statesm,np.array([0, 1, 2, 0, 1, 1, 0, 0, 0])))

0
345


In [77]:
""""as a reminder, our board setup looks like:
A tic tac toe board looks like a 3x3 grid:
0 1 2 
3 4 5
6 7 8
But we can reinterpret it as a 1 dimensional array of size 9.
0 1 2 3 4 5 6 7 8
"""
#Initiate q-table with zeroes
qtable = np.zeros((19683, 9))
print(qtable)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [157]:
class QAgent(Agent):
  def __init__(self, side):
    self.side = side
    self.qtable = np.zeros((19683, 9)) #q-table - we'll update this whenever a win or loss is reached.
    #+5 to reward wins, -5 to indicate losses
    self.qlookup = generateStates()
    #Define Gamma, Epilson, Alpha   
    self.gamma = 0.95 
    self.start_epsilon = 1.0 
    self.epsilon = 1.0 
    self.min_epsilon = 0.01 
    self.decay_rate = 0.05 
    self.alpha = 0.8                

  def reset_epsilon(self):
     self.epsilon = self.start_epsilon

  def decay_epsilon(self):
    if(self.epsilon >= self.min_epsilon and not (self.epsilon - self.decay_rate <self.min_epsilon)):
      self.epsilon = self.epsilon - self.decay_rate
  #reset ALL modifiable properties.
  def reset_mod_properties(self):
    self.qtable = np.zeros((19683, 9)) #q-table - we'll update this whenever a win or loss is reached.=
    self.epsilon = self.start_epsilon  # Current exploration rate
    
  def move(self, tictactoe):
    #our move action will have to be a bit more complicated than just picking random valid moves.
    #mode 1 - randomly select a valid move    
    """
    select action a
      with probability epsilon select random action a
    """
    if(random.uniform(0, 1)<=self.epsilon): #random check falls within range of 0,epsilon, select randomly
    #if(1==2): #DEBUG
      empty = tictactoe.getempty()
      tictactoe.place1d(self.side, random.choice(empty))
    else: #else peform a q-learn action
    #mode 2 - get all valid moves, select a move then find the corresponding Q-table value,
    #and update it.
      """
        else selection action a = argmax(Q(s,a'))
      """
      empty = tictactoe.getempty() #get all valid actions
      argument_values = []
      for desired_space in empty:
        hypo_state = tictactoe.state.copy()
        hypo_state[desired_space] = self.side #check what the next action would be for each state
        targetindex = searchStates(self.qlookup, hypo_state)
        argument_values.append(self.qtable[targetindex][desired_space]) #get the qtable's current value and add it to argument values
      best_action_index = np.argmax(argument_values)
      """
        carry out action a
        We don't actually execute the action yet on the main tictactoe board
        because we still want to observe s and s'
      """
      best_action = empty[best_action_index]

      #instead we create a new hypothetical, and observe how this one would go.
      action_hypo = tictactoe.state.copy()
      action_hypo[best_action] = self.side

      """
      class GAMESTATE(Enum):
          ONGOING = 0 
          PLAYER_1 = 1
          PLAYER_2 = 2
          DRAW = 3 
          ERROR = 4 (shouldn't be possible.)
      """

      """
      observe next state s' and reward r

      here we check if s' as a result of our action
      is a victory or defeat.

      r value table: 
      If victory, +5, 
      if ongoing, +0,
      if draw, -1 (we want to avoid draws but they're not the end of the world.)
      if defeat or any other value (error), -5 (we want to avoid this as much as possible)
      """
      vict = tictactoe.victory_check() 
      reward = 0
      possiblerewards = []
      #we need to do this check manually because we need to account for what side the player's on
      if vict == 0: #indicates the game is still ongoing, we need to do one more check for successive states
        reward = 0
        tictactoe2 = TicTacToe(action_hypo) #create a hypothetical tictactoe
        empty2 = tictactoe2.getempty() #get all valid actions
        if(len(empty2) == 0): #if the board's full, don't look-ahead in s', instead check victory state
          vict2 = tictactoe.victory_check()
          if vict2 == 3: #Draw - game over
            possiblerewards.append(-1)
          elif vict2 == self.side: #Win - game over
            possiblerewards.append(5)
          else: #a defeat or an error state - game over
            possiblerewards.append(-5)
        else:
          #print("DEBUG", empty2)
          #print("DEBUG", tictactoe2.printstate_xo())
          for desired_space2 in empty2:
            copy3 = tictactoe2.state.copy()
            tictactoe3 = TicTacToe(copy3)
            tictactoe3.place1d(self.side, desired_space2)
            vict3 = tictactoe3.victory_check() 
            if vict3 == 0: #the game is still ongoing 2 actions in, so we'll just note that it's still going.
              possiblerewards.append(0)
            elif vict == 3:
              possiblerewards.append(-1)
            elif vict == self.side: #a win
              possiblerewards.append(5)
            else: #a defeat or another error-ish value
              possiblerewards.append(-5)
      elif vict == 3: #Draw - game over
        reward = -1
        possiblerewards.append(0)
      elif vict == self.side: #Win - game over
        reward = 5
        possiblerewards.append(0)
      else: #a defeat or an error state - game over
        reward = -5
        possiblerewards.append(0)

      """
      calculate Q_target = r+gamma*max[Q(s',A)]
      """
      Q_target = reward + (self.gamma * max(possiblerewards))
      """
      calculate Q_delta = Q_target - Q(s,a)
      add alpha*Q_delta to Q(s,a)
      modify Q(s,a)
      """
      qsa_index = searchStates(self.qlookup, action_hypo)

      Q_delta = Q_target - self.qtable[qsa_index][best_action]
      self.qtable[qsa_index][best_action] = self.qtable[qsa_index][best_action] + (self.alpha * Q_delta) 
      
      #print("DEBUG", self.qtable[qsa_index])
      """
      s=s'
      advance the state.
      """
      tictactoe.place1d(self.side, best_action)
    self.decay_epsilon()
  

In [123]:
QAgent1 = QAgent(1)
QAgent2 = QAgent(2)

In [124]:
a.reset_board()
QAgent1.move(a)
a.printstate()

[0 0 0]
[0 0 0]
[1 0 0]


In [125]:
a.reset_board()
QAgent1.move(a)
QAgent2.move(a)
QAgent1.move(a)
QAgent2.move(a)
a.printstate()

[2 0 1]
[0 0 2]
[1 0 0]


#QAgent Games

Games are formed as P1 vs P2

##QAgent vs Random Player

In [162]:
episodes = 201       # Total episodes - i.e. the number of games we'll play to completion
#38s for 100 episodes
#~6min for 1k episodes

#reset the agents
QAgent1 = QAgent(1)
QAgent2 = QAgent(2)

randage1 = RandomAgent(1)
randage2 = RandomAgent(2)

#player1wins,player2wins,drawwins, errors
results_array = [0,0,0,0]

#make a new tictactoe game
tictactournament = TicTacToe()
for i in range(0, episodes):
  gameend = False
  result = 4 
  tictactournament.reset_board()
  if(i%5 == 0):
    print("Episode ", i)
  if(i%100 == 0):
    print("Game of Episode ", i)
  while (gameend == False):
    QAgent1.move(tictactournament)
    if(i%100 == 0):
      print("P1")
      tictactournament.printstate_xo()
    vict = tictactournament.victory_check() 
    if vict !=0:
      result = vict
      gameend = True
      break
    randage2.move(tictactournament)
    if(i%100 == 0):
      print("P2")
      tictactournament.printstate_xo()
    vict = tictactournament.victory_check() 
    if vict !=0:
      result = vict
      gameend = True
      break
  results_array[result-1]+=1

print("Game Statistics: P1 Win, P2 Win, Draw, Error")  
print(results_array)

Episode  0
Game of Episode  0
P1
E E E  
E E E  
E X E  
P2
E O E  
E E E  
E X E  
P1
E O E  
E X E  
E X E  
P2
O O E  
E X E  
E X E  
P1
O O X  
E X E  
E X E  
P2
O O X  
E X E  
O X E  
P1
O O X  
X X E  
O X E  
P2
O O X  
X X O  
O X E  
P1
O O X  
X X O  
O X X  
Episode  5
Episode  10
Episode  15
Episode  20
Episode  25
Episode  30
Episode  35
Episode  40
Episode  45
Episode  50
Episode  55
Episode  60
Episode  65
Episode  70
Episode  75
Episode  80
Episode  85
Episode  90
Episode  95
Episode  100
Game of Episode  100
P1
X E E  
E E E  
E E E  
P2
X E E  
E O E  
E E E  
P1
X X E  
E O E  
E E E  
P2
X X E  
O O E  
E E E  
P1
X X X  
O O E  
E E E  
Episode  105
Episode  110
Episode  115
Episode  120
Episode  125
Episode  130
Episode  135
Episode  140
Episode  145
Episode  150
Episode  155
Episode  160
Episode  165
Episode  170
Episode  175
Episode  180
Episode  185
Episode  190
Episode  195
Episode  200
Game of Episode  200
P1
X E E  
E E E  
E E E  
P2
X E E  
E O E  
E E 

In [163]:
#check to make sure epsilon actually decreased
print(QAgent1.epsilon)

0.049999999999999684


##Random Player vs QAgent

In [164]:
#we'll swap the starting player to see if has any effect
#player1wins,player2wins,drawwins, errors
results_array = [0,0,0,0]

#make a new tictactoe game
tictactournament = TicTacToe()
for i in range(0, episodes):
  gameend = False
  result = 4 
  tictactournament.reset_board()
  if(i%5 == 0):
    print("Episode ", i)
  if(i%100 == 0):
    print("Game of Episode ", i)
  while (gameend == False):
    randage1.move(tictactournament)
    if(i%100 == 0):
      print("P1")
      tictactournament.printstate_xo()
    vict = tictactournament.victory_check() 
    if vict !=0:
      result = vict
      gameend = True
      break
    QAgent2.move(tictactournament)
    if(i%100 == 0):
      print("P2")
      tictactournament.printstate_xo()
    vict = tictactournament.victory_check() 
    if vict !=0:
      result = vict
      gameend = True
      break
  results_array[result-1]+=1

print("Game Statistics: P1 Win, P2 Win, Draw, Error")  
print(results_array)

Episode  0
Game of Episode  0
P1
E E E  
E E E  
E E X  
P2
E E E  
O E E  
E E X  
P1
E X E  
O E E  
E E X  
P2
O X E  
O E E  
E E X  
P1
O X E  
O E E  
X E X  
P2
O X E  
O E E  
X O X  
P1
O X E  
O E X  
X O X  
P2
O X E  
O O X  
X O X  
P1
O X X  
O O X  
X O X  
Episode  5
Episode  10
Episode  15
Episode  20
Episode  25
Episode  30
Episode  35
Episode  40
Episode  45
Episode  50
Episode  55
Episode  60
Episode  65
Episode  70
Episode  75
Episode  80
Episode  85
Episode  90
Episode  95
Episode  100
Game of Episode  100
P1
E E E  
E E E  
E E X  
P2
O E E  
E E E  
E E X  
P1
O X E  
E E E  
E E X  
P2
O X O  
E E E  
E E X  
P1
O X O  
E E E  
X E X  
P2
O X O  
O E E  
X E X  
P1
O X O  
O X E  
X E X  
P2
O X O  
O X E  
X O X  
P1
O X O  
O X X  
X O X  
Episode  105
Episode  110
Episode  115
Episode  120
Episode  125
Episode  130
Episode  135
Episode  140
Episode  145
Episode  150
Episode  155
Episode  160
Episode  165
Episode  170
Episode  175
Episode  180
Episode  185
Ep

##QAgent vs QAgent

In [165]:
#Lastly we'll have 2 QAgents play vs each other

QAgent3 = QAgent(1)
QAgent4 = QAgent(2)
#we'll swap the starting player to see if has any effect
#player1wins,player2wins,drawwins, errors
results_array = [0,0,0,0]

#make a new tictactoe game
tictactournament = TicTacToe()
for i in range(0, episodes):
  gameend = False
  result = 4 
  tictactournament.reset_board()
  if(i%5 == 0):
    print("Episode ", i)
  if(i%100 == 0):
    print("Game of Episode ", i)
  while (gameend == False):
    QAgent3.move(tictactournament)
    if(i%100 == 0):
      print("P1")
      tictactournament.printstate_xo()
    vict = tictactournament.victory_check() 
    if vict !=0:
      result = vict
      gameend = True
      break
    QAgent4.move(tictactournament)
    if(i%100 == 0):
      print("P2")
      tictactournament.printstate_xo()
    vict = tictactournament.victory_check() 
    if vict !=0:
      result = vict
      gameend = True
      break
  results_array[result-1]+=1

print("Game Statistics: P1 Win, P2 Win, Draw, Error")  
print(results_array)

Episode  0
Game of Episode  0
P1
E E E  
E X E  
E E E  
P2
E E O  
E X E  
E E E  
P1
E E O  
E X E  
E X E  
P2
E E O  
O X E  
E X E  
P1
E E O  
O X E  
E X X  
P2
E O O  
O X E  
E X X  
P1
X O O  
O X E  
E X X  
Episode  5
Episode  10
Episode  15
Episode  20
Episode  25
Episode  30
Episode  35
Episode  40
Episode  45
Episode  50
Episode  55
Episode  60
Episode  65
Episode  70
Episode  75
Episode  80
Episode  85
Episode  90
Episode  95
Episode  100
Game of Episode  100
P1
X E E  
E E E  
E E E  
P2
X O E  
E E E  
E E E  
P1
X O X  
E E E  
E E E  
P2
X O X  
O E E  
E E E  
P1
X O X  
O X E  
E E E  
P2
X O X  
O X O  
E E E  
P1
X O X  
O X O  
X E E  
Episode  105
Episode  110
Episode  115
Episode  120
Episode  125
Episode  130
Episode  135
Episode  140
Episode  145
Episode  150
Episode  155
Episode  160
Episode  165
Episode  170
Episode  175
Episode  180
Episode  185
Episode  190
Episode  195
Episode  200
Game of Episode  200
P1
X E E  
E E E  
E E E  
P2
X O E  
E E E  
E E 

#Conclusion

When playing vs random agents, our Q-learning agents performed slightly worse in the player 1 seat, but better in the player 2 seat, and were able to draw less often on the whole. We speculate that the struggle to win more in the player 1 seat is likely due to the relatively short number of episodes we had to train the Q-learning agent in due to time/usage constraints and that further adjustments could be made to the hyperparameters to help it more quickly learn the game. 

https://en.wikipedia.org/wiki/First-player_and_second-player_win

If both players were playing perfectly, Tic-Tac-Toe should always end in a draw. However, our agents are not perfect - the random agent picks randomly, and the Q-learning agent is trying to learn the game from scratch. 

https://momath.org/wp-content/uploads/2021/08/Alyssa-Choi-Tic-Tac-Toe.pdf

https://www.quora.com/How-can-we-win-in-Tic-Tac-Toe-when-we-go-second

In the event that players are playing imperfectly, an advantage can be gained by the first player by taking as many the four corners of the board as possible (this is demonstrated by Choi et al as the "fork strategy"). We observe that the Q Agent when played in the "1st player seat" learned this advantage over the course of its training and always attempted to claim a corner in its first move once trained, and attempted to make moves to claim the other corners especially when playing vs the other Q Agent.

In the event that the first player is trying the forking strategy, it is generally in player 2's best interest to claim the center of the board. It appears that our Q Agent in the player 2 seat did generally not learn this strategy - against the random agent, which was less likely to employ that strategy, it instead attempted to claim the corners as much as possible instead (possibly to try and replicate the forking strategy), and when facing a Q Agent in the player 1 seat that had learned the forking strategy, it went for more short-sighted attempts to block any three-way connections. This is likely also another situation where more training episodes and more time to experiment with different hyperparameters could lead to interesting results, particularly in seeing if a Player 2 Q Agent in a Q agent-only game could learn the strategy of takign the center space to block Player 1 attempting to fork.